In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
gdf = gpd.read_parquet("merged_data.parquet").rename(columns={
        'LSOA code (2021)': 'LSOA code',
        'date': 'period',
        'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)': 'imd_rank',
        'Burglaries amount': 'burglaries'
    })

In [ ]:

def downcast_numeric(df: pd.DataFrame) -> pd.DataFrame:
    # 1. downcast integers
    int_cols = df.select_dtypes(include=['int64']).columns
    df[int_cols] = (
        df[int_cols]
        .apply(pd.to_numeric, downcast='integer')
    )
    
    # 2. downcast floats
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = (
        df[float_cols]
        .apply(pd.to_numeric, downcast='float')
    )

    return df

# Usage:
gdf["LSOA code"] = gdf["LSOA code"].astype("category")

gdf = downcast_numeric(gdf)
gdf.info()

In [ ]:
static = {
   "Education locations", "Emergency locations", "Entertainment locations",
   "Food locations", "Leisure locations", "Parking locations", "Shopping locations",
   "Public transport locations", "Dwelling type|Flat, maisonette or apartment (%)",
   "Ethnic Group|Asian/Asian British (%)", "Ethnic Group|BAME (%)", "Ethnic Group|Black/African/Caribbean/Black British (%)",
   "Ethnic Group|Mixed/multiple ethnic groups (%)", "Ethnic Group|Other ethnic group (%)",
   "Ethnic Group|White (%)", "Household Composition|% Couple household with dependent children",
   "Household Composition|% Couple household without dependent children",
   r"Household Composition|% Lone parent household",
    "Household Composition|% One person household",
    "Household Composition|% Other multi person household",
    "Households|All households", "Tenure|Owned outright (%)",
    "Tenure|Owned with a mortgage or loan (%)", "Tenure|Private rented (%)",
    "Tenure|Social rented (%)", "Car or van availability|1 car or van in household (%)",
    "Car or van availability|2 cars or vans in household (%)",
    "Car or van availability|3 cars or vans in household (%)",
    "Car or van availability|4 or more cars or vans in household (%)",
    "Car or van availability|Cars per household",
    "Car or van availability|No cars or vans in household (%)",
    "Public Transport Accessibility Levels|% 0-1 (poor access)|Level3_65",       
    "Public Transport Accessibility Levels|% 2-3 (average access)|Level3_66",       
    "Public Transport Accessibility Levels|% 4-6 (good access)|Level3_67",       
    "Public Transport Accessibility Levels|Average Score|Level3_64",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|0",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|1a",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|1b",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|2",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|3",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|4",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|5",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|6a",       
    "Public Transport Accessibility Levels|Number of people in each PTAL level:|6b",
}

dynamic = set(gdf.drop(columns=["period", "geometry", "LSOA code", "burglaries"]).columns) - set(static)

In [ ]:
t0 = gdf['period'].min()
# convert each period to the number of months since t0
time_idx = (
    (gdf['period'].dt.year  - t0.year) * 12
  + (gdf['period'].dt.month - t0.month)
)
# center & scale
gdf['time_s'] = (time_idx - time_idx.mean()) / time_idx.std()
gdf['month_sin'] = np.sin(2*np.pi*gdf['period'].dt.month/12)
gdf['month_cos'] = np.cos(2*np.pi*gdf['period'].dt.month/12)
# Temporal trend
gdf['prev_month']  = gdf.groupby('LSOA code', observed=True)['burglaries'].shift(1)
gdf['prev_year'] = gdf.groupby('LSOA code', observed=True)['burglaries'].shift(12)

In [ ]:
from scipy.sparse import csr_matrix

def create_length_weighted_adjacency(gdf: gpd.GeoDataFrame):
    """
    From a GeoDataFrame with columns ['LSOA code','geometry'],
    build:
      - W: an N×N sparse matrix where W[i,j] = length of shared border between i and j
      - D: the diagonal degree‐matrix with D[i,i] = sum_j W[i,j]
      - idx_map: mapping LSOA code -> integer index i in [0..N-1]
    """
    # 1) Extract one geometry per LSOA and fix geometry validity
    geo = (
        gdf[['LSOA code','geometry']]
        .drop_duplicates('LSOA code')
        .reset_index(drop=True)
    )
    geo['geometry'] = geo['geometry'].buffer(0)

    # 2) Build index mapping
    codes   = geo['LSOA code'].tolist()
    idx_map = {code: idx for idx, code in enumerate(codes)}
    N       = len(codes)

    rows, cols, data = [], [], []

    # 3) Loop over each LSOA and its touching neighbours
    for i, row in geo.iterrows():
        code_i, geom_i = row['LSOA code'], row['geometry']
        # find those that touch
        touching = geo[geo.geometry.touches(geom_i)]
        for _, nbr in touching.iterrows():
            j = idx_map[nbr['LSOA code']]
            # only handle each pair once (i<j)
            if i >= j:
                continue
            geom_j = nbr['geometry']
            # compute shared boundary length
            shared = geom_i.intersection(geom_j)
            length = shared.length
            if length > 0:
                # add both directions
                rows += [i, j]
                cols += [j, i]
                data += [length, length]

    # 4) Build sparse W and degree‐matrix D
    W = csr_matrix((data, (rows, cols)), shape=(N, N), dtype=float)
    deg = np.array(W.sum(axis=1)).ravel()
    D = csr_matrix((deg, (np.arange(N), np.arange(N))), shape=(N, N))

    return W, D, idx_map


W, D, idx_map = create_length_weighted_adjacency(gdf)

In [ ]:
matrix = gdf.dropna(subset=["prev_month", "prev_year"]).copy()
matrix['lsoa_idx'], cats = pd.factorize(matrix['LSOA code'])
matrix = matrix.drop(columns=["geometry", "period", "LSOA code"])
# TODO: proper scaling of all parameters if needed
# for col in matrix.drop(columns=["lsoa_idx"]).columns:
# # for col in matrix.drop(columns=["lsoa_idx", "burglaries", "month_sin", "month_cos", "prev_year", "prev_month"]).columns:
#     matrix[col] = np.log1p(matrix[col])
#     matrix[col] = (matrix[col] - matrix[col].mean())/matrix[col].std()

matrix.info()

In [ ]:
seasonal = {
    "month_sin", "month_cos"
}
time_trend = {
    "time_s"
}
temporal = {
    "prev_month", "prev_year"
}
set(matrix.columns) - static - dynamic - seasonal - time_trend - temporal


In [ ]:
matrix.describe()

In [ ]:
matrix.min().sort_values()

# Training the model

In [ ]:
# # ──────────────────────────────────────────────────────────────────────────────
# # Cell 1: Imports & Data Preparation
# # ──────────────────────────────────────────────────────────────────────────────
# import numpy as np
# import jax.numpy as jnp
# from jax import random, device_put
# import numpyro
# import numpyro.distributions as dist
# from numpyro.infer import SVI, Trace_ELBO, Predictive
# from numpyro.infer.autoguide import AutoNormal
# from numpyro.infer.initialization import init_to_feasible
# from numpyro.optim import Adam

# # Assume you’ve already got in your namespace:
# #   • matrix        : pandas.DataFrame with columns
# #       'lsoa_idx','burglaries',
# #       static_cols (e.g. ['static_1','static_2','static_3']),
# #       dynamic_cols (e.g. ['dynamic_1','dynamic_2','dynamic_3']),
# #       'time_s','month_sin','month_cos','prev_month'
# #   • static_cols   : list of static column names
# #   • dynamic_cols  : list of dynamic column names
# #   • W             : (N×N) NumPy array of shared-border weights

# static_cols = list(static)
# dynamic_cols = list(dynamic)
# # 1) Convert adjacency to JAX & build degree matrix
# W_jax = device_put(jnp.array(W.toarray()))                     # [N×N]
# D_jax = device_put(jnp.diag(D.toarray()))                      # [N×N]

# # 2) Static matrix [N × P_static]
# static_df = (
#     matrix[['lsoa_idx'] + static_cols]
#     .drop_duplicates('lsoa_idx')
#     .sort_values('lsoa_idx')
# )
# static_matrix = device_put(
#     jnp.array(static_df[static_cols].values)
# )

# # 3) Dynamic matrix [n_obs × P_dynamic]
# dynamic_matrix = device_put(
#     jnp.array(matrix[dynamic_cols].values)
# )

# # 4) Vectorize other columns
# vecs = {
#     'lsoa_idx': device_put(jnp.array(matrix['lsoa_idx'].values)),   # [n_obs]
#     'counts':   device_put(jnp.array(matrix['burglaries'].values)), # [n_obs]
#     'time_s':   device_put(jnp.array(matrix['time_s'].values)),     # [n_obs]
#     'sin':      device_put(jnp.array(matrix['month_sin'].values)),  # [n_obs]
#     'cos':      device_put(jnp.array(matrix['month_cos'].values)),  # [n_obs]
#     'lag1':     device_put(jnp.array(matrix['prev_month'].values)),  # [n_obs]
# }


In [ ]:
# # ──────────────────────────────────────────────────────────────────────────────
# # Cell 2: Model Definition & SVI Function
# # ──────────────────────────────────────────────────────────────────────────────

# def hierarchical_nb_full(
#     lsoa_idx, counts, n_lsoas,
#     static_matrix, n_static,
#     time_s, sin, cos, lag1,
#     dynamic_matrix, n_dynamic,
#     W, D
# ):
#     """
#     Hierarchical NB with:
#       • static covariates
#       • continuous trend & seasonality
#       • one‐month self‐lag
#       • dynamic covariates
#       • CAR spatial random effect
#     """
#     # 1) Static effects
#     sigma_s    = numpyro.sample('sigma_static', dist.HalfNormal(1.0))
#     beta_s     = numpyro.sample(
#         'beta_static',
#         dist.Normal(0., sigma_s).expand([n_static])
#     )
#     static_eff = (static_matrix[lsoa_idx] * beta_s).sum(-1)

#     # 2) Time trend
#     beta_time = numpyro.sample('beta_time', dist.Normal(0., 0.25))
#     time_eff  = beta_time * time_s

#     # 3) Seasonality
#     beta_sin  = numpyro.sample('beta_sin', dist.Normal(0., 0.25))
#     beta_cos  = numpyro.sample('beta_cos', dist.Normal(0., 0.25))
#     seas_eff  = beta_sin * sin + beta_cos * cos

#     # 4) Self‐lag
#     beta_lag1 = numpyro.sample('beta_lag1', dist.Normal(0., 0.25))
#     lag_eff   = beta_lag1 * lag1

#     # 5) Dynamic covariates
#     tau_d      = numpyro.sample('tau_dyn',  dist.HalfNormal(0.5))
#     beta_d     = numpyro.sample(
#         'beta_dyn',
#         dist.Normal(0., tau_d).expand([n_dynamic])
#     )
#     dyn_eff    = (dynamic_matrix * beta_d).sum(-1)

#     # 6) Spatial CAR random effect
#     tau_u      = numpyro.sample('tau_u', dist.HalfNormal(1.0))
#     rho        = numpyro.sample('rho',   dist.Uniform(0., 1.0))
#     Q          = tau_u * (D - rho * W)
#     u_raw      = numpyro.sample(
#         'u_raw',
#         dist.MultivariateNormal(jnp.zeros(n_lsoas), precision_matrix=Q)
#     )
#     u          = u_raw - jnp.mean(u_raw)
#     spatial_re = u[lsoa_idx]

#     # 7) Intercept & overdispersion
#     alpha = numpyro.sample('alpha', dist.Normal(jnp.log(12.8), 0.25))
#     phi   = numpyro.sample('phi',   dist.Gamma(2., 1.0))

#     # 8) Linear predictor & likelihood
#     log_mu = (alpha
#             + static_eff
#             + time_eff
#             + seas_eff
#             + lag_eff
#             + dyn_eff
#             + spatial_re)
#     mu = jnp.exp(log_mu)

#     numpyro.sample('y_obs',
#                    dist.NegativeBinomial2(phi, mu),
#                    obs=counts)


# def run_svi_full(
#     static_matrix, dynamic_matrix, W, D, vecs,
#     n_lsoas, n_static, n_dynamic,
#     num_steps=3000, lr=5e-5, seed=0
# ):
#     # 1) Build data dict
#     data = {
#         'n_lsoas':        n_lsoas,
#         'static_matrix':  static_matrix,
#         'n_static':       n_static,
#         'dynamic_matrix': dynamic_matrix,
#         'n_dynamic':      n_dynamic,
#         'W':              W,
#         'D':              D,
#         **vecs,
#     }

#     # 2) Prior predictive check
#     print("Running prior predictive check…")
#     prior   = Predictive(hierarchical_nb_full, params=None, num_samples=200)
#     pd_draws= prior(random.PRNGKey(seed), **data)['y_obs']
#     lower   = np.percentile(pd_draws, 3,  axis=0)
#     upper   = np.percentile(pd_draws, 97, axis=0)
#     pd_cov  = ((data['counts'] >= lower) & (data['counts'] <= upper)).mean()
#     print(f"Prior coverage (94% CI): {pd_cov:.2%}")

#     # 3) SVI setup
#     guide = AutoNormal(hierarchical_nb_full, init_loc_fn=init_to_feasible)
#     svi   = SVI(hierarchical_nb_full, guide, Adam(lr), Trace_ELBO())
#     rng   = random.PRNGKey(seed)
#     state = svi.init(rng, **data)

#     # 4) Optimization loop
#     for i in range(num_steps):
#         rng, _ = random.split(rng)
#         state, loss = svi.update(state, **data)
#         if i % 500 == 0:
#             print(f"Step {i:>4d} loss = {loss:.1f}")

#     # 5) Return fitted guide & params
#     return svi.get_params(state), guide, data


In [ ]:
# # ──────────────────────────────────────────────────────────────────────────────
# # Cell 3: Run the SVI
# # ──────────────────────────────────────────────────────────────────────────────
# params, guide, data_dict = run_svi_full(
#     static_matrix, dynamic_matrix, W_jax, D_jax, vecs,
#     n_lsoas   = matrix['lsoa_idx'].nunique(),
#     n_static  = len(static_cols),
#     n_dynamic = len(dynamic_cols),
#     num_steps = 500,
#     lr        = 5e-5,
#     seed      = 42
# )


In [ ]:
# import torch
# import pyro
# import pyro.distributions as dist
# from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS
# from pyro.optim import ClippedAdam
# from pyro.infer.autoguide import AutoDiagonalNormal
# # 1) Select device
# pyro.clear_param_store()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.set_default_device(device)
# print("Using device:", device)                        # should print "cuda" :contentReference[oaicite:0]{index=0}

# # 2) Generate synthetic data on GPU
# true_loc = 1.0
# true_scale = 2.0
# N = 5000
# data = dist.Normal(true_loc, true_scale).sample([N]).to(device)

# # 3) Define model with parameters on GPU
# def model(data):
#     # Prior on the mean
#     mu = pyro.sample("mu", dist.Normal(torch.tensor(0., device=device), torch.tensor(5., device=device)).to_event(0)) 
#     # Prior on the scale (must be positive)
#     sigma = pyro.sample("sigma", dist.HalfCauchy(torch.tensor(2., device=device)).to_event(0))
#     with pyro.plate("data", data.shape[0]):
#         # Likelihood: observe data points
#         pyro.sample("obs", dist.Normal(mu, sigma), obs=data)

# # 4) Define a simple AutoGuide on GPU
# guide = AutoDiagonalNormal(model)

# # 5) Set up SVI with a GPU-backed optimizer
# svi = SVI(model, guide, ClippedAdam({"lr": 0.02}), loss=Trace_ELBO())

# # 6) Run VI for a few hundred steps
# num_steps = 1000
# for step in range(num_steps):
#     loss = svi.step(data)
#     if step % 100 == 0:
#         print(f"Step {step} \t ELBO loss = {loss:.1f}")  # This loop should run on GPU :contentReference[oaicite:1]{index=1}

# # 7) Draw posterior samples with NUTS on GPU
# nuts_kernel = NUTS(model)
# mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200)
# mcmc.run(data)                                        # runs on GPU, provided all tensors are on cuda :contentReference[oaicite:2]{index=2}

# # 8) Examine posterior summaries
# posterior = mcmc.get_samples()
# print({k: v.mean().item() for k, v in posterior.items()})


# Pyro test

In [ ]:
import pandas as pd

# assume matrix_df is your full DataFrame, with a Period‐like “period” column:
#   columns: ['period','lsoa_idx','burglaries', *all your 64 features*, ...]

# 3) split into training vs. forecast
#    – matrix: all rows before the last month
#    – next_month: only the last month’s rows
matrix      = matrix[matrix['time_s'] <  matrix['time_s'].max()].copy()
next_month  = matrix[matrix['time_s'] == matrix['time_s'].max()].copy()

print("Training months:", matrix['time_s'].min(), "→", matrix['time_s'].max())
print("Forecast month:", next_month['time_s'].unique())

In [ ]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import ClippedAdam
from pyro.infer.autoguide import AutoDiagonalNormal

# ──────────────────────────────────────────────────────────────────────────────
# 0) GPU + Pyro setup
pyro.clear_param_store()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

# ──────────────────────────────────────────────────────────────────────────────
# 1) load_and_prepare now takes a DataFrame `df` already in memory
def load_and_prepare(df):
    # a) extract target & index
    y = df["burglaries"].astype(np.int32).values
    idx = df["lsoa_idx"].astype(np.int64).values

    # b) build feature matrix from all cols except these two
    feat_cols = [c for c in df.columns if c not in ("burglaries", "lsoa_idx")]
    X = df[feat_cols].values.astype(np.float32)

    # c) drop rows with any NaN/inf
    mask = np.isfinite(X).all(axis=1)
    X, y, idx = X[mask], y[mask], idx[mask]

    # d) standardize features
    means = X.mean(axis=0)
    stds  = X.std(axis=0) + 1e-6
    X = (X - means) / stds

    # e) move to GPU tensors
    return {
        "lsoa_idx": torch.tensor(idx, dtype=torch.long,   device=device),
        "X":        torch.tensor(X,   dtype=torch.float32,device=device),
        "y":        torch.tensor(y,   dtype=torch.int32,  device=device),
        "n_lsoas":  int(idx.max()) + 1,
        "n_attrs":  X.shape[1],
        "feat_cols": feat_cols,
        "means": means,
        "stds": stds,
    }

data = load_and_prepare(matrix)

# ──────────────────────────────────────────────────────────────────────────────
# 2) Hierarchical Poisson model (unchanged)
def burglary_model(lsoa_idx, X, y=None, use_subsample=True):
    n_lsoas = data["n_lsoas"]
    n_attrs = data["n_attrs"]

    mu_a    = pyro.sample("mu_a",    dist.Normal(0., 1.0))
    sigma_a = pyro.sample("sigma_a", dist.HalfNormal(1.0))

    with pyro.plate("ls", n_lsoas):
        a = pyro.sample("a", dist.Normal(mu_a, sigma_a))

    b_attr = pyro.sample(
        "b_attr",
        dist.Normal(0., 1.0).expand([n_attrs]).to_event(1)
    )

    N = lsoa_idx.shape[0]

    if use_subsample and y is not None:
        with pyro.plate("data", size=N, subsample_size=2048) as i:
            eta = a[lsoa_idx[i]] + (X[i] * b_attr).sum(-1)
            mu  = torch.exp(eta.clamp(-10, 10))
            pyro.sample("obs", dist.Poisson(mu), obs=y[i])
    else:
        with pyro.plate("data", N):
            eta = a[lsoa_idx] + (X * b_attr).sum(-1)
            mu  = torch.exp(eta.clamp(-10, 10))
            pyro.sample("obs", dist.Poisson(mu), obs=y if y is not None else None)

# ──────────────────────────────────────────────────────────────────────────────
# 3) Fit with SVI
guide = AutoDiagonalNormal(burglary_model)
svi   = SVI(burglary_model, guide, ClippedAdam({"lr":1e-2}), loss=Trace_ELBO())

for step in range(500):
    loss = svi.step(data["lsoa_idx"], data["X"], data["y"])
    if step % 100 == 0:
        print(f"Step {step:4d}  ELBO loss = {loss:.1f}")


# Prediction

In [36]:
next_data = load_and_prepare(next_month)
print("Training columns:", data["feat_cols"])
print("Prediction columns:", next_data["feat_cols"])

from pyro.infer import Predictive

predictive = Predictive(
    model=lambda lidx, x: burglary_model(lidx, x, y=None, use_subsample=False),
    guide=guide,
    num_samples=1000,
    return_sites=["obs"]
)

# This returns a dict of tensors; obs has shape [1000, num_locations]
pred_samples = predictive(next_data["lsoa_idx"], next_data["X"])


Training columns: ['imd_rank', 'Income Rank (where 1 is most deprived)', 'Employment Rank (where 1 is most deprived)', 'Education, Skills and Training Rank (where 1 is most deprived)', 'Health Deprivation and Disability Rank (where 1 is most deprived)', 'Crime Rank (where 1 is most deprived)', 'Barriers to Housing and Services Rank (where 1 is most deprived)', 'Living Environment Rank (where 1 is most deprived)', 'Education locations', 'Emergency locations', 'Entertainment locations', 'Food locations', 'Leisure locations', 'Parking locations', 'Shopping locations', 'Public transport locations', 'Dwelling type|Flat, maisonette or apartment (%)', 'Ethnic Group|Asian/Asian British (%)', 'Ethnic Group|BAME (%)', 'Ethnic Group|Black/African/Caribbean/Black British (%)', 'Ethnic Group|Mixed/multiple ethnic groups (%)', 'Ethnic Group|Other ethnic group (%)', 'Ethnic Group|White (%)', 'Household Composition|% Couple household with dependent children', 'Household Composition|% Couple household 

RuntimeError: The size of tensor a (68) must match the size of tensor b (64) at non-singleton dimension 1
Trace Shapes:          
 Param Sites:          
Sample Sites:          
    mu_a dist      |   
        value      |   
 sigma_a dist      |   
        value      |   
      ls dist      |   
        value 4994 |   
       a dist 4994 |   
        value 4994 |   
  b_attr dist      | 64
        value      | 64
    data dist      |   
        value 4994 |   

In [ ]:
pred_samples

In [ ]:
pred_mean = pred_samples["obs"].float().mean(dim=0)
pred_mean

In [ ]:
lower = pred_samples["obs"].kthvalue(int(0.05 * 1000), dim=0).values
upper = pred_samples["obs"].kthvalue(int(0.95 * 1000), dim=0).values

In [ ]:
next_month["pred_mean"]    = pred_mean.cpu().numpy()
next_month["pred_5pct"]    = lower.cpu().numpy()
next_month["pred_95pct"]   = upper.cpu().numpy()
next_month["predicted"] = (
    (next_month["burglaries"] > next_month["pred_5pct"])
    & 
    (next_month["burglaries"] < next_month["pred_95pct"])
)

In [ ]:
next_month[["lsoa_idx", "burglaries", "pred_mean", "pred_5pct", "pred_95pct", "predicted"]]

In [ ]:
# 1. Faster optimizer
svi = SVI(burglary_model, guide, ClippedAdam({"lr": 5e-2}), loss=Trace_ELBO())

# 2. Profile to measure gains
%timeit svi.step(data["lsoa_idx"], data["X"], data["y"])


# 3. Check for batching possibility (if dataset is large)
with pyro.plate("data", len(y), subsample_size=512) as i:
    pyro.sample("obs", dist.Poisson(mu[i]), obs=y[i])
